In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [3]:
# !pip install wordcloud
from wordcloud import WordCloud

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
data = pd.read_excel("отзывы за лето.xls")

In [9]:
data.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [10]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
 !pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from string import punctuation

     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 8.2MB 22.5MB/s 


In [12]:
russian_stopwords = stopwords.words("russian")

In [13]:
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [14]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

In [15]:
train_corpus = " ".join(data["text"])
train_corpus = train_corpus.lower()

In [16]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
tokens = word_tokenize(train_corpus)

In [18]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [19]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [20]:
tokens_filtered_top[:10]

['приложение',
 'всё',
 'и',
 'очень',
 'удобно',
 'в',
 'я',
 'на',
 'работать',
 'удобный']

In [21]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [22]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train, x_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [25]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in x_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in x_test], dtype=np.int32)

In [ ]:
x_train.shape

(15798, 40)

In [ ]:
x_test.shape

(3950, 40)

In [ ]:
x_train[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  8, 20], dtype=int32)

### Keras model

In [26]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [27]:
num_classes = 2
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

**Keras model со слоем  tf.keras.layers.Embedding по умолчанию**

In [28]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation('softmax'))

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 128)           25600     
_________________________________________________________________
conv1d (Conv1D)              (None, 38, 128)           49280     
_________________________________________________________________
activation (Activation)      (None, 38, 128)           0         
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [30]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 21s 171ms/step - loss: 0.6305 - accuracy: 0.7489 - val_loss: 0.4090 - val_accuracy: 0.8430


In [ ]:
score = model.evaluate(x_train, y_train, batch_size=batch_size, verbose=1)
print('\n')
print('Train score:', score[0])
print('Train accuracy:', score[1])

31/31 [==============================] - 1s 38ms/step - loss: 0.4003 - accuracy: 0.8469


Test score: 0.40030211210250854
Test accuracy: 0.8468793630599976


In [ ]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

8/8 [==============================] - 1s 38ms/step


In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 37ms/step - loss: 0.4024 - accuracy: 0.8468


Test score: 0.40237054228782654
Test accuracy: 0.8468354344367981


**Keras model со слоем  tf.keras.layers.Embedding с предобученными векторами**

In [32]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-07-19 06:10:35--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-07-19 06:10:35--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-07-19 06:10:35--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [35]:
import os
import pathlib

In [36]:
path_to_glove_file = os.path.join("glove.6B.100d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [37]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(data['text']).batch(128)
vectorizer.adapt(text_ds)

In [38]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [39]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 730 words (12057 misses)


In [40]:
model = Sequential()
model.add(Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation('softmax'))

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1278900   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         64128     
_________________________________________________________________
activation_3 (Activation)    (None, None, 128)         0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [42]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [43]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 45s 67ms/step - loss: 0.5639 - accuracy: 0.7959 - val_loss: 0.4733 - val_accuracy: 0.8677


In [44]:
score = model.evaluate(x_train, y_train, batch_size=batch_size, verbose=1)
print('\n')
print('Train score:', score[0])
print('Train accuracy:', score[1])

31/31 [==============================] - 0s 11ms/step - loss: 0.4601 - accuracy: 0.8713


Train score: 0.4601086378097534
Train accuracy: 0.8713127970695496


In [45]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

8/8 [==============================] - 0s 24ms/step


In [46]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 5ms/step - loss: 0.4790 - accuracy: 0.8603


Test score: 0.47899946570396423
Test accuracy: 0.8602531552314758


Модель с предобученными векторами получилась лучше, accuracy на test равна 0.86, а у модели со слоем tf.keras.layers.Embedding по умолчанию accuracy на test - 0.84